<a href="https://colab.research.google.com/github/DivyaNarayan0613/DivyaNarayan0613/blob/main/PyBulletRobotic_arm(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This PyBullet script sets up a simulation with a Franka Panda robot, a table, a tray, and a random object. The robot performs a sequence of movements to pick up the object.

This program simulates a Franka Panda robotic arm in PyBullet, making it pick up an object from a table using its gripper.

1. Importing Libraries
   
os: Used for handling file paths.

pybullet: The physics simulation engine.

pybullet_data: Contains predefined URDF (Universal Robot Description Format) models.

math: Used for angle calculations (e.g., math.pi for π).

time: Used to slow down the simulation for stability.

2. Connecting to PyBullet

p.connect(p.GUI): Starts the PyBullet physics engine in GUI mode.
    
pybullet_data.getDataPath(): Gets the directory path where predefined robot models (URDF files) are stored.

3. Loading the Simulation Environment
Loads the Franka Panda robot, table, tray, and random object into the simulation

useFixedBase=True: Ensures the robotic arm does not move.

4. Setting Gravity

Sets the gravity force in the Z-direction (downwards).

5. Setting Up the Camera
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55, -0.35, 0.2])

Positions the simulation camera for better visualization.

6. Defining Time Control Variables

control_dt = 1/240: Defines the simulation time step (frames per second).

p.setTimeStep(control_dt): Ensures simulation consistency.

7. State Machine for Robot Movements
   
state_durations: List defining how long each movement step should last.

state_t: Tracks time spent in the current state.

current_state: Tracks which movement step the robot is in.

8. Main Simulation Loop
   
Runs continuously until the user stops it.

Updates time tracking (state_t).

p.configureDebugVisualizer(p.COV_ENABLE_SINGLE_STEP_RENDERING) ensures smooth rendering.




9. Movement Stages of the Robot

The robot moves through 4 states to pick up an object.

State 0: Move Arm to Initial Position

Moves the robot arm to an initial pre-grasp position.

Opens the gripper (joint 9 and joint 10).

State 1: Adjust Arm Position to Align with Object - Moves the arm slightly downward towards the object.

State 2: Close Gripper to Pick Up Object - Closes the gripper tightly to grasp the object.

State 3: Lift Object Up - Lifts the object upwards after grasping.





10. Transition to the Next State
Moves to the next state when state_t exceeds the duration for the current step.
    
Resets to state 0 after completing all states.

11. Simulate Each Step & Add Delay

p.stepSimulation() advances the physics simulation.

time.sleep(control_dt) slows down the loop for stability.

In [ ]:
import os
import pybullet as p
import pybullet_data
import math
import time  # Import time for delay

# Connect to PyBullet
p.connect(p.GUI)
urdfRootPath = pybullet_data.getDataPath()

# Load URDF models
pandaUid = p.loadURDF(os.path.join(urdfRootPath, "franka_panda/panda.urdf"), useFixedBase=True)
tableUid = p.loadURDF(os.path.join(urdfRootPath, "table/table.urdf"), basePosition=[0.5, 0, -0.65])
trayUid = p.loadURDF(os.path.join(urdfRootPath, "tray/traybox.urdf"), basePosition=[0.65, 0, 0])
objectUid = p.loadURDF(os.path.join(urdfRootPath, "random_urdfs/000/000.urdf"), basePosition=[0.7, 0, 0.1])

# Set gravity
p.setGravity(0, 0, -10)

# Camera setup
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55, -0.35, 0.2])

# Time step and state control
control_dt = 1.0 / 240.0
p.setTimeStep(control_dt)

state_durations = [1, 1, 1, 1]  # Duration for each state
state_t = 0.0
current_state = 0

while True:
    state_t += control_dt
    p.configureDebugVisualizer(p.COV_ENABLE_SINGLE_STEP_RENDERING)

    if current_state == 0:  # Move arm to initial grasp position
        p.setJointMotorControl2(pandaUid, 0, p.POSITION_CONTROL, 0)
        p.setJointMotorControl2(pandaUid, 1, p.POSITION_CONTROL, math.pi / 4.0)
        p.setJointMotorControl2(pandaUid, 2, p.POSITION_CONTROL, 0)
        p.setJointMotorControl2(pandaUid, 3, p.POSITION_CONTROL, -math.pi / 2.0)
        p.setJointMotorControl2(pandaUid, 4, p.POSITION_CONTROL, 0)
        p.setJointMotorControl2(pandaUid, 5, p.POSITION_CONTROL, 3 * math.pi / 4.0)
        p.setJointMotorControl2(pandaUid, 6, p.POSITION_CONTROL, -math.pi / 4.0)
        p.setJointMotorControl2(pandaUid, 9, p.POSITION_CONTROL, 0.08)  # Open gripper
        p.setJointMotorControl2(pandaUid, 10, p.POSITION_CONTROL, 0.08)

    elif current_state == 1:  # Slight adjustment for gripping
        p.setJointMotorControl2(pandaUid, 1, p.POSITION_CONTROL, math.pi / 4.0 + 0.15)
        p.setJointMotorControl2(pandaUid, 3, p.POSITION_CONTROL, -math.pi / 2.0 + 0.15)

    elif current_state == 2:  # Close gripper (grasp object)
        p.setJointMotorControl2(pandaUid, 9, p.POSITION_CONTROL, 0.0, force=200)
        p.setJointMotorControl2(pandaUid, 10, p.POSITION_CONTROL, 0.0, force=200)

    elif current_state == 3:  # Lift object up
        p.setJointMotorControl2(pandaUid, 1, p.POSITION_CONTROL, math.pi / 4.0 - 1.0)
        p.setJointMotorControl2(pandaUid, 3, p.POSITION_CONTROL, -math.pi / 2.0 - 1.0)

    # Transition to the next state
    if state_t > state_durations[current_state]:
        current_state += 1
        if current_state >= len(state_durations):
            current_state = 0  # Restart cycle
        state_t = 0

    p.stepSimulation()
    time.sleep(control_dt)  # Add delay for stable simulation


error: Not connected to physics server.